**Unsupervised 1**

# K-Means Clustering - Part 2

## Learning objectives

- In the first example, we introduced K-means clustering by choosing 5 clusters. 
- **But**, how do we know 5 (or any k) is the right number? Maybe 3 would be sufficient, or a higher number would reveal more nuanced patterns.
- Now, we'll apply the **elbow method** and calculate **silhouette scores** determine optimal cluster numbers 


<br>

We'll continue the London crimes MSOA example.

**Setup: Import required libraries**

In [15]:
import pandas as pd
import altair as alt
import numpy as np

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

---

<br>
<br>

### Step 1: Load and explore the data

`ldn_crimes_msoa.csv` contains month crime totals by type for all MSOA (census boundaries for areas of between 5000-1500 people) areas in London, covering the period September 2022 to 2025. See [here](https://www.ons.gov.uk/methodology/geography/ukgeographies/statisticalgeographies) for ONS explanation.

In [3]:
data = pd.read_csv('clean-data/ldn_crimes_msoa.csv')                    # Crime counts by MSOA
# data = pd.read_csv('clean-data/ldn_crimes_agg_202209_202509.csv')     # For crimes by LSOA

print(f"\nWe have {data['MSOA code'].nunique():,} MSOA neighborhoods")
print(f"Dataset shape: {data.shape}\n")
data.head(8)  # Print first few rows


We have 1,002 MSOA neighborhoods
Dataset shape: (395267, 5)



,MSOA code,MSOA name,Month,Crime type,count
0,E02000001,City of London 001,2022-09,Anti-social behaviour,10
1,E02000001,City of London 001,2022-09,Bicycle theft,2
2,E02000001,City of London 001,2022-09,Burglary,3
3,E02000001,City of London 001,2022-09,Criminal damage and arson,2
4,E02000001,City of London 001,2022-09,Drugs,2
5,E02000001,City of London 001,2022-09,Other theft,37
6,E02000001,City of London 001,2022-09,Public order,4
7,E02000001,City of London 001,2022-09,Robbery,6


<br>
<br>

### Step 2: Prepare Data for Clustering

**First**, transform into feature matrix with format:
- Each row = one neighbourhood (MSOA)
- Each column = one crime type
- Values = how much of that crime happens there (we could choose average monthly totals, sum etc)

In [4]:
# Create a pivot table: neighbourhoods x crime types
crime_matrix = data.pivot_table(
    index=['MSOA code'],
    columns='Crime type',
    values='count',
    fill_value=0,       # Fill any missing values with 0
    aggfunc='sum'        # Sum counts over time
)
crime_matrix.head(3)

Crime type,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences
MSOA code,,,,,,,,,,,,,,
E02000001,186,118,137,122,92,12,1445,7,163,165,498,1086,111,551
E02000002,681,7,63,137,241,29,163,15,145,45,29,23,300,880
E02000003,741,17,170,246,113,40,265,14,208,136,123,62,410,1067


<br>

**Second**, scale the data using StandardScaler

In [6]:
# 1. Convert to proportion (what % of area's crime is each type?), each row sums to 1
crime_matrix_percentages = crime_matrix.div(crime_matrix.sum(axis=1), axis=0).fillna(0)  # Fill any NaNs with 0 

# 2. Standardise the data (important for K-means)
scaler = StandardScaler()
crime_matrix_scaled = scaler.fit_transform(crime_matrix_percentages)

# 3. Convert back to DataFrame for easier handling (since fit_transform returns a NumPy array)
crime_matrix_scaled_df = pd.DataFrame(
    crime_matrix_scaled,
    index=crime_matrix_percentages.index,
    columns=crime_matrix_percentages.columns
)

print("\nScaled crime profiles:")
crime_matrix_scaled_df.head(3)


Scaled crime profiles:


Crime type,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences
MSOA code,,,,,,,,,,,,,,
E02000001,-3.632102,1.151576,-1.315957,-1.930876,-1.108070,-0.972943,7.040934,-1.471408,-1.570150,0.948195,1.008432,4.251980,-1.777317,-2.380416
E02000002,0.598932,-0.878499,-1.661411,-0.407855,3.417239,-0.007473,-0.846996,0.568878,0.166845,-0.804684,-0.783266,-0.666857,0.043207,1.231694
E02000003,-0.253636,-0.683764,-0.343595,0.777798,-0.328179,0.060387,-0.394725,-0.238913,0.654742,1.180161,-0.342123,-0.472245,0.144498,0.807769


<br>
<br>

### Step 3: Finding the Optimal Number of Clusters

So far, we chose 5 clusters arbitrarily. But how do we know if 5 is the right number? Maybe 3 would be simpler, or 7 would reveal more nuanced patterns?

This is a fundamental question in unsupervised learning: **without labels to guide us, how do we know we've found the "right" groupings?**

We'll explore two methods to answer this question.

<br>
<br>

#### Method 1: The Elbow Method

The **Elbow Method** measures how tightly grouped our clusters are. As we add more clusters, the groupings naturally get tighter (in the extreme case, if every point is its own cluster, the tightness is perfect). 

We look for the "elbow" - the point where adding more clusters stops giving us much benefit.

In [ ]:
# Calculate within-cluster sum of squares (WCSS) for different K values
print("Testing different numbers of clusters...")
print("-" * 50)

wcss_scores = []
K_range = range(2, 11)      # Choose a range of K values to test

for k in K_range:
    # Fit K-means with k clusters
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)       # Method is same as before, just with a loop over K values
    kmeans.fit(crime_matrix_scaled_df)
    
    # Store the WCSS (also called inertia)
    wcss_scores.append(kmeans.inertia_)
    
    print(f"K={k}: Within-cluster sum of squares = {kmeans.inertia_:,.0f}")

# Create the elbow plot
elbow_data = pd.DataFrame({
    'K': list(K_range),
    'WCSS': wcss_scores
})
elbow_data

Testing different numbers of clusters...
--------------------------------------------------
K=2: Within-cluster sum of squares = 11,739
K=3: Within-cluster sum of squares = 10,589
K=4: Within-cluster sum of squares = 9,693
K=5: Within-cluster sum of squares = 9,057
K=6: Within-cluster sum of squares = 8,477
K=7: Within-cluster sum of squares = 8,066
K=8: Within-cluster sum of squares = 7,618
K=9: Within-cluster sum of squares = 7,365
K=10: Within-cluster sum of squares = 7,129


,K,WCSS
0,2,11739.276049
1,3,10589.034587
2,4,9693.297313
3,5,9056.743315
4,6,8476.771761
5,7,8066.407084
6,8,7617.987959
7,9,7364.616497
8,10,7129.335682


<br>

Plot elbow data

In [11]:
elbow_plot = alt.Chart(elbow_data).mark_line(point=True, color='steelblue').encode(
    x=alt.X('K:Q').scale(domain=[2, 10]).axis(tickCount=9, title='Number of Clusters (K)'),
    y=alt.Y('WCSS:Q').scale(zero=False).axis(format=',d').title('Within-Cluster Sum of Squares'),
    tooltip=[alt.Tooltip('K:Q', title='Clusters'), alt.Tooltip('WCSS:Q', format=',.0f', title='WCSS')]
).properties(
    width=500,
    height=300,
    title='The Elbow method: finding optimal K'
)

# Add a reference line at K=5
rule = alt.Chart(pd.DataFrame({'K': [5]})).mark_rule(
    color='red',
    strokeDash=[5, 5]
).encode(
    x='K:Q',
    size=alt.value(2)
)

# Add annotation
text = alt.Chart(pd.DataFrame({
    'K': [5],
    'WCSS': [elbow_data[elbow_data['K']==5]['WCSS'].values[0]],
    'label': ['Potential elbow']
})).mark_text(
    align='left',
    dx=10,
    dy=-10,
    color='red',
    fontSize=12
).encode(
    x='K:Q',
    y='WCSS:Q',
    text='label:N'
)

elbow_plot + rule + text

alt.LayerChart(...)

<br>

##### Interpreting the elbow plot

Look for where the line "bends" like an elbow:
- **Before the elbow**: Adding clusters significantly improves grouping (steep drop)
- **After the elbow**: Diminishing returns (line flattens out)
- **At the elbow**: Good balance between simplicity and accuracy

In our case, the elbow isn't well defined, but somewhere in the range k=3-5 looks good.

<br>
<br>
<br>
<br>

### Method 2: Silhouette Analysis

While the elbow method looks at compactness, the **Silhouette Score** measures how distinct our clusters are from each other. 

Score interpretation:
- **Close to 1**: Clusters are well-separated and distinct
- **Around 0.25-0.5**: Reasonable structure exists
- **Below 0.25**: Clusters overlap (common in real-world data!)
- **Negative**: Points might be in the wrong cluster

<br>

> **Note on real-world data**: Crime patterns often overlap between neighbourhoods - there's no "perfect" separation. Low scores don't mean failure; they reflect the continuous nature of crime patterns. Choosing a lower (e.g. LSOA) or higher (e.g. borough) level of statistical geography would affect this further.

In [ ]:
# Calculate Silhouette scores for different K values
print("\nMeasuring cluster separation quality...")
print("-" * 50)

silhouette_scores = []

for k in K_range:
    # Fit K-means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)       # Method is same as before, just with a loop over K values
    cluster_labels = kmeans.fit_predict(crime_matrix_scaled_df)
    
    # Calculate silhouette score
    score = silhouette_score(crime_matrix_scaled_df, cluster_labels)    # Use Scikit-learn's silhouette_score function
    silhouette_scores.append(score)

    
    print(f"K={k}: Silhouette Score = {score:.3f}")


Measuring cluster separation quality...
--------------------------------------------------
K=2: Silhouette Score = 0.192
K=3: Silhouette Score = 0.126
K=4: Silhouette Score = 0.134
K=5: Silhouette Score = 0.133
K=6: Silhouette Score = 0.135
K=7: Silhouette Score = 0.117
K=8: Silhouette Score = 0.117
K=9: Silhouette Score = 0.118
K=10: Silhouette Score = 0.113


,K,Silhouette
0,2,0.192405
1,3,0.126380
2,4,0.133937
3,5,0.132744
4,6,0.135358
5,7,0.116721
6,8,0.116534
7,9,0.117525
8,10,0.113001


Our silhouette scores are all below 0.2, which might seem concerning at first. However, for crime data at this low MSOA level it could be understandable:
1. Crime patterns exist on a SPECTRUM, not in distinct boxes
2. Neighbouring areas influence each other (crime doesn't stop at boundaries)
3. Many neighbourhoods have "mixed" profiles that could reasonably fit multiple clusters

This doesn't mean our clustering is wrong - it means we're finding patterns in naturally overlapping data.

<br>

> **Real data is messy:**
> - Low scores don't mean bad analysis
> - Real patterns are often fuzzy
> - Domain knowledge becomes crucial for interpretation

In [20]:
# Add our scores to a dataframe
silhouette_data = pd.DataFrame({
    'K': list(K_range),
    'Silhouette': silhouette_scores
})
# Find the best relative score
best_k = K_range[np.argmax(silhouette_scores)]
best_score = max(silhouette_scores)
print(f"\nBest relative separation: K={best_k} with score={best_score:.3f}")


Best relative separation: K=2 with score=0.192


In [30]:
silhouette_plot = alt.Chart(silhouette_data).mark_line(point=True, color='green').encode(
    x=alt.X('K:Q').scale(domain=[2, 10]).axis(tickCount=9).title('Number of Clusters (K)'),
    y=alt.Y('Silhouette:Q').title('Silhouette Score').scale(domain=[0.09, max(silhouette_scores) * 1.1]),
    tooltip=[alt.Tooltip('K:Q', title='Clusters'), alt.Tooltip('Silhouette:Q', format='.3f', title='Score')]
).properties(
    width=400,
    height=250,
    title=alt.TitleParams(
        text='Silhouette analysis',
        subtitle='Cluster separation quality - higher is better')
)

best_point = alt.Chart(pd.DataFrame({
    'K': [best_k],
    'Silhouette': [best_score],
    'label': [f'Best: K={best_k}']
})).mark_point(
    color='red',
    size=100
).encode(
    x='K:Q',
    y='Silhouette:Q',
    tooltip=['label:N']
)

silhouette_plot + best_point

alt.LayerChart(...)

So, based on our Silhouette score, `K=2` is the best number cluster to maximuse separation quality. However, having just two clusters likely **oversimplifies**.

> Somewhere in K=4-6 should offer more meaningful patterns.

<br>
<br>
<br>

#### Checking profiles

Combining the insight from the Elbow plot and Silhouette scores suggests k=4 or k=5 are likely the best choices.

In [ ]:
# Check if our clusters align with intuitive expectations
print("CLUSTER VALIDATION CHECK")
print("=" * 60)

chosen_k = 5 # (or change to chosen value)

kmeans = KMeans(n_clusters=chosen_k, random_state=12, n_init=10)
clusters = kmeans.fit_predict(crime_matrix_scaled_df)

crime_matrix_percentages['Cluster'] = clusters
cluster_profiles = crime_matrix_percentages.groupby('Cluster').mean()

# Also get the overall London average for comparison, and differences from London average
london_average = crime_matrix_percentages.drop('Cluster', axis=1).mean()
differences = cluster_profiles.subtract(london_average, axis=1)

# Check logical patterns
print("Do clusters show distinct primary crime types? (difference from London msoa average)") 
for i in range(chosen_k):
    # top_crime = cluster_profiles.iloc[i].max()
    # print(f"  Cluster {i}: {cluster_profiles.iloc[i].idxmax()} ({top_crime:.1%})")

    # Get largest difference value irrespective of sign
    top_crime_difference_pos = differences.iloc[i].max()
    top_crime_difference_neg = differences.iloc[i].min()
    print(f"  Cluster {i}: {differences.iloc[i].idxmax() if top_crime_difference_pos > abs(top_crime_difference_neg) else differences.iloc[i].idxmin()} (+{top_crime_difference_pos if top_crime_difference_pos > abs(top_crime_difference_neg) else top_crime_difference_neg:.1%})")

print("\nAre cluster sizes reasonable? (not too imbalanced)")
sizes = pd.Series(clusters).value_counts().sort_index()
print(f"  Size range: {sizes.min()} to {sizes.max()} neighbourhoods")
print(f"  Ratio: 1:{sizes.max()/sizes.min():.1f}")

print("\nDo similar crime types group together?")
theft_crimes = ['Other theft', 'Theft from the person', 'Shoplifting']
for i in range(chosen_k):
    theft_total = cluster_profiles.loc[i, theft_crimes].sum()
    if theft_total > 0.3:  # 30% or more
        print(f"  Cluster {i}: High theft concentration ({theft_total:.1%})")



CLUSTER VALIDATION CHECK
Do clusters show distinct primary crime types? (difference from London msoa average)
  Cluster 0: Anti-social behaviour (+3.7%)
  Cluster 1: Violence and sexual offences (+5.8%)
  Cluster 2: Shoplifting (+8.5%)
  Cluster 3: Theft from the person (+12.8%)
  Cluster 4: Vehicle crime (+5.1%)

Are cluster sizes reasonable? (not too imbalanced)
  Size range: 70 to 300 neighbourhoods
  Ratio: 1:4.3

Do similar crime types group together?
  Cluster 3: High theft concentration (39.5%)


---

<br>

### Insight: Statistics vs. Domain Knowledge

This exercise demonstrates an important machine learning lesson:

**Statistical metrics are guides, not judges.** Our low silhouette scores don't indicate failure - they reveal the true nature of crime patterns:
- Crime types blend gradually between areas
- Socioeconomic factors create continuums, not categories  
- Real-world phenomena rarely fit into neat boxes

<br>

**What matters for policy:**
1. Can we identify meaningful patterns? Yes
2. Do clusters suggest different interventions? Yes
3. Are the groupings stable and reproducible? Yes

<br>

The "messiness" of real data is a feature, not a bug - it teaches us to combine statistical analysis with domain expertise.